In [6]:
# coding: utf-8
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
warnings.filterwarnings('ignore')

In [20]:
data=pd.read_csv("F:/data.csv")
data

,user,label,sex,age,provider,level,verified,using_time,regist_type,card_a_cnt,...,ip_nun_mean.1,ip_nun_std,ip_hour_nun_max,ip_hour_nun_min,ip_hour_nun_mean,ip_hour_nun_std,ip_min_nun_max,ip_min_nun_min,ip_min_nun_mean,ip_min_nun_std
0,Train_00000,0.0,1,24871,1,2,0,24712,1,24712,...,1.500000,0.577350,2.0,1.0,1.25000,0.452267,2.0,1.0,1.055556,0.232311
1,Train_00001,1.0,1,24889,1,3,0,24716,1,24719,...,1.000000,0.000000,1.0,1.0,1.00000,0.000000,1.0,1.0,1.000000,0.000000
2,Train_00002,0.0,1,24963,1,3,0,24736,7,24712,...,1.000000,NaN,1.0,1.0,1.00000,NaN,1.0,1.0,1.000000,0.000000
3,Train_00003,0.0,1,24840,3,3,0,24719,3,24712,...,1.909091,0.943880,2.0,1.0,1.03125,0.176777,1.0,1.0,1.000000,0.000000
4,Train_00004,0.0,1,24871,3,2,0,24707,3,24712,...,1.000000,NaN,1.0,1.0,1.00000,NaN,1.0,1.0,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96197,TestB_19010,NaN,1,24865,1,3,0,24740,7,24712,...,1.500000,0.707107,1.0,1.0,1.00000,0.000000,1.0,1.0,1.000000,0.000000
96198,TestB_12466,NaN,1,24908,1,3,0,24736,7,24712,...,1.000000,0.000000,1.0,1.0,1.00000,0.000000,1.0,1.0,1.000000,0.000000
96199,TestB_05971,NaN,1,24993,1,2,1,24728,1,24706,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96200,TestB_16084,NaN,1,24908,1,3,0,24711,3,24719,...,1.500000,0.707107,1.0,1.0,1.00000,0.000000,1.0,1.0,1.000000,0.000000


In [7]:
def kfold_stats_feature(train, test, feats, k):
    folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)  # 这里最好和后面模型的K折交叉验证保持一致

    train['fold'] = None
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['label'])):
        train.loc[val_idx, 'fold'] = fold_

    kfold_features = []
    for feat in feats:
        nums_columns = ['label']
        for f in nums_columns:
            colname = feat + '_' + f + '_kfold_mean'
            kfold_features.append(colname)
            train[colname] = None
            for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['label'])):
                tmp_trn = train.iloc[trn_idx]
                order_label = tmp_trn.groupby([feat])[f].mean()
                tmp = train.loc[train.fold == fold_, [feat]]
                train.loc[train.fold == fold_, colname] = tmp[feat].map(order_label)
                # fillna
                global_mean = train[f].mean()
                train.loc[train.fold == fold_, colname] = train.loc[train.fold == fold_, colname].fillna(global_mean)
            train[colname] = train[colname].astype(float)

        for f in nums_columns:
            colname = feat + '_' + f + '_kfold_mean'
            test[colname] = None
            order_label = train.groupby([feat])[f].mean()
            test[colname] = test[feat].map(order_label)
            # fillna
            global_mean = train[f].mean()
            test[colname] = test[colname].fillna(global_mean)
            test[colname] = test[colname].astype(float)
    del train['fold']
    return train, test

In [8]:
def lgb_model(train, target, test, k):
    feats = [f for f in train.columns if f not in ['user', 'label']]
    print('Current num of features:', len(feats))
    folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)
    oof_probs = np.zeros(train.shape[0])
    output_preds = 0
    offline_score = []
    feature_importance_df = pd.DataFrame()
    parameters = {
        'learning_rate': 0.05,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 63,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'min_data_in_leaf': 20,
        'verbose': -1,
        'nthread': 8
    }

    for i, (train_index, test_index) in enumerate(folds.split(train, target)):
        train_y, test_y = target[train_index], target[test_index]
        train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]

        dtrain = lgb.Dataset(train_X,
                             label=train_y)
        dval = lgb.Dataset(test_X,
                           label=test_y)
        lgb_model = lgb.train(
                parameters,
                dtrain,
                num_boost_round=5000,
                valid_sets=[dval],
                early_stopping_rounds=100,
                verbose_eval=100,
        )
        oof_probs[test_index] = lgb_model.predict(test_X[feats], num_iteration=lgb_model.best_iteration)
        offline_score.append(lgb_model.best_score['valid_0']['auc'])
        output_preds += lgb_model.predict(test[feats], num_iteration=lgb_model.best_iteration)/folds.n_splits
        print(offline_score)
        # feature importance
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = lgb_model.feature_importance(importance_type='gain')
        fold_importance_df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('OOF-MEAN-AUC:%.6f, OOF-STD-AUC:%.6f' % (np.mean(offline_score), np.std(offline_score)))
    print('feature importance:')
    print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(15))
    return output_preds, oof_probs, np.mean(offline_score),feature_importance_df

In [21]:
data['city_level'] = data['city'].map(str) + '_' + data['level'].map(str)
data['city_balance_avg'] = data['city'].map(str) + '_' + data['balance_avg'].map(str)
train = data[~data['label'].isnull()].copy()
target = train['label']
test = data[data['label'].isnull()].copy()

target_encode_cols = ['province', 'city', 'city_level', 'city_balance_avg']
train, test = kfold_stats_feature(train, test, target_encode_cols, 5)
train.drop(['city_level', 'city_balance_avg'], axis=1, inplace=True)
test.drop(['city_level', 'city_balance_avg'], axis=1, inplace=True)

lgb_preds, lgb_oof, lgb_score, fea_df = lgb_model(train=train, target=target, test=test, k=5)

sub_df = test[['user']].copy()
sub_df['prob'] = lgb_preds
#sub_df.to_csv('F:/sub.csv', index=False)
# fea_df.to_csv('F:/feature.csv',index=False)

Current num of features: 832
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's auc: 0.737829
[200]	valid_0's auc: 0.73701
Early stopping, best iteration is:
[106]	valid_0's auc: 0.738497
[0.7384967813610117]
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's auc: 0.735191
[200]	valid_0's auc: 0.734821
Early stopping, best iteration is:
[114]	valid_0's auc: 0.735624
[0.7384967813610117, 0.7356241080457369]
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's auc: 0.72694
[200]	valid_0's auc: 0.726645
Early stopping, best iteration is:
[145]	valid_0's auc: 0.72903
[0.7384967813610117, 0.7356241080457369, 0.7290300473967176]
Train until valid scores didn't improve in 100 rounds.
[100]	valid_0's auc: 0.728103
[200]	valid_0's auc: 0.729544
Early stopping, best iteration is:
[192]	valid_0's auc: 0.729653
[0.7384967813610117, 0.7356241080457369, 0.7290300473967176, 0.7296533083380494]
Train until valid scores didn't improve in 100 rou

In [28]:
sub_df.sort_values(by="user")[24315:].to_csv("F:/sub.csv",index=False)